BerkeleyX: Data8.3x

Foundations of Data Science: Prediction and Machine Learning

In [ ]:
from datascience import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Lec 4.1 Prediction

Prediction

In [ ]:
galton = Table.read_table('../../data/galton.csv')

heights = Table().with_column(
    'MidParent', galton.column('midparentHeight'),
    'Child', galton.column('childHeight'))
heights

In [ ]:
heights.scatter('MidParent')

In [ ]:
def predict_child(x):
    chosen = heights.where('MidParent', are.between(x - 0.5, x + 0.5))
    return np.average(chosen.column('Child'))

In [ ]:
predictions = heights.apply(predict_child, 'MidParent')

In [ ]:
heights = heights.with_column(
    'Original Prediction', predictions
)

heights

In [ ]:
heights.scatter('MidParent')

Lec 4.2 Linear Regression

In [ ]:
def regression_line(r):
    x = np.random.normal(0, 1, 10000)
    z = np.random.normal(0, 1, 10000)
    y = r*x + (np.sqrt(1-r**2))*z
    plt.figure(figsize=(7, 7))
    plt.xlim(-4, 4)
    plt.ylim(-4, 4)
    plt.scatter(x, y, s=10)
    plt.plot([-4, 4], [-4*r,4*r], color='g', lw=2)
    if r >= 0:
        plt.plot([-4,4],[-4,4], lw=2, color='r')
    else:
        plt.plot([-4,4], [4,-4], lw=2, color='r')

In [ ]:
regression_line(.6)
regression_line(.9)
regression_line(.2)
regression_line(0)

Regression estimate:

- convert x to SU
- multiply the result by r
- convert the result back from SU

In [ ]:
# example: Adult Mountain Lions
# length mean is 84 inches, SD 8 inches
# weight mean is 125 pounds, SD 15 pounds
# correlation .6
# if weight is 155 the estimated length is?
x = (155 - 125) / 15 # (w - w_m) / sd_w
y = .6 * x           # r * x
len = y * 8 + 84     # y * sd_l + l_m
x, y, len

Lec 4.3 Regression to the Mean

Lec 4.4 Regression Equation

In SU:
- it passes from (0, 0)
- y = r * x

In actual units:
- it passes from (avg_x, avg_y)
- its slope r_x is SD_y / SD_x

Formula:

- (estimate of y - avg of y) / (SD of y) = r * (given x - avg of x) / (SD of x)
- estimate of y = slope * x + intercept

where:
- slope = r * (SD of y) / (SD of x)
- intercept = (avg of y) - slope * (avg of y)

In [ ]:
def standard_units(x):
    """x is a list"""
    return (x - np.average(x))/np.std(x)

def correlation(t, x, y):
    """table t, columns labeled x and y"""
    x_su = standard_units(t.column(x))
    y_su = standard_units(t.column(y))
    return np.average(x_su * y_su)

def slope(t, x, y):
    """table t, columns labeled x (given) and y (predicted)"""
    r = correlation(t, x, y)
    return r * np.std(t.column(y))/np.std(t.column(x))

def intercept(t, x, y):
    """table t, columns labeled x and y"""
    a = slope(t, x, y)
    return np.average(t.column(y)) - a*np.average(t.column(x))

In [ ]:
galton_slope = slope(heights, 'MidParent', 'Child')
galton_intercept = intercept(heights, 'MidParent', 'Child')
galton_slope, galton_intercept

In [ ]:
sample = heights.take(123)
print(sample)
print('linear prediction:', galton_slope * sample.column(0)[0] + galton_intercept)

In [ ]:
def fitted_values(t, x, y):
    a = slope(t, x, y)
    b = intercept(t, x, y)
    return a * t.column(x) + b

In [ ]:
regression_predictions = fitted_values(heights, 'MidParent', 'Child')

In [ ]:
heights = heights.with_column(
    'Regression Prediction', regression_predictions
)
heights

In [ ]:
heights.scatter('MidParent')

Lec 4.5 Interpreting the Slope

Reading and Practice for Section 4

Drink water price vs rain precipitation
- price avg: .919016
- price SD: .464763
- rain avg: 1010.4
- rain SD: 752.475
- r: .262079

In [ ]:
# correlation(water, ‘mm_precipitation’, ‘OBS_VALUE’)
r = .262079

# np.average(t.column(y)) - a*np.average(t.column(x))
intercept = 1010.4


# x is precipitation
x = 700
slope = r * (.464763) / (752.475)
print(slope)
intercept = ( .919016) - slope * (1010.4)
print(intercept)

# y is estimated price
y = slope * x + intercept
print(y)